Surname generation

In [1]:
import os

download_name = "surnames_with_splits.csv.bz2"
if not os.path.exists(download_name):
    import requests
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/nlp_repo/main/lectures/a05/{download_name}")
    with open(download_name, "wb") as fp:
        fp.write(response.content)
    response.close()
        
name = "surnames_with_splits.csv"
if not os.path.exists(name):
    import bz2
    with open(download_name, 'rb') as bzf, open(name, 'wb') as fp:
        fp.write(bz2.decompress(bzf.read()))       

In [2]:
import os

import torch
import torch.nn.functional as F

from argparse import Namespace
from tqdm.notebook import tqdm

from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Settings

In [3]:
args = Namespace(
    # Data and path information
    surname_csv="surnames_with_splits.csv",
    model_filename="model.pth",
    # Model hyper parameter
    char_embedding_size=32,
    rnn_hidden_size=32,
    # Training hyper parameter
    num_epochs=100,
    learning_rate=0.001,
    batch_size=128,
    seed=1337,
    early_stop=10,
    # Runtime hyper parameter
)

args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Reading

In [4]:
df = pd.read_csv(args.surname_csv).drop(["nationality_index"], axis=1)
df


,nationality,split,surname
0,Arabic,train,Totah
1,Arabic,train,Abboud
2,Arabic,train,Fakhoury
3,Arabic,train,Srour
4,Arabic,train,Sayegh
...,...,...,...
10975,Vietnamese,test,Dinh
10976,Vietnamese,test,Phung
10977,Vietnamese,test,Quang
10978,Vietnamese,test,Vu


## Vocabulary

Generalized vocabulary can have:
* padding token - to fill up empty space
* unknown token - token for out-of-vocabulary tokens
* begin sequence - token for start of a sequence
* end sequence - token for end of a sequence


In [5]:
class Vocab(object):

    def __init__(self, tokens=None, pad_token=None, unk_token=None, begin_seq_token=None, end_seq_token=None):
        self._tok2idx = {}
        self._idx2tok = {}
        
        self.pad_token = pad_token
        self.pad_idx = None
        if pad_token is not None:
            self.pad_idx = self.add_token(pad_token)
        
        self.unk_token = unk_token
        self.unk_idx = None
        if unk_token is not None:
            self.unk_idx = self.add_token(unk_token)

        self.begin_seq_token = begin_seq_token
        self.begin_seq_idx = None
        if begin_seq_token is not None:
            self.begin_seq_idx = self.add_token(begin_seq_token)

        self.end_seq_token = end_seq_token
        self.end_seq_idx = None
        if end_seq_token is not None:
            self.end_seq_idx = self.add_token(end_seq_token)

        if tokens is not None:
            self.add_tokens(tokens)

    def add_token(self, token):
        if token not in self._tok2idx:
            idx = len(self._tok2idx)
            self._tok2idx[token] = idx
            self._idx2tok[idx] = token
            return idx
        return self._tok2idx[token]

    def add_tokens(self, tokens):
        return [self.add_token(token) for token in tokens]

    def ordered_indices(self):
        return sorted(self._idx2tok)

    def ordered_tokens(self):
        for i in sorted(self._idx2tok):
            yield self._idx2tok[i]

    def __getitem__(self, token_or_idx):
        if isinstance(token_or_idx, str):
            return self._tok2idx.get(token_or_idx, self.unk_idx)
        if isinstance(token_or_idx, int):
            return self._idx2tok.get(token_or_idx, self.unk_token)

    def __len__(self):
        return len(self._tok2idx)

    def __iter__(self):
        for i in sorted(self._idx2tok):
            yield self._idx2tok[i]

    def info(self):
        txt = f"Vocabulary size:{len(self)}"
        for i in range(min(4, len(self))):
            txt += f" {self[i]}:{i}"
        txt += " ..."
        print(txt)

chars = {ch for surname in df[df.split == "train"].surname for ch in surname}
surname_vocab = Vocab(sorted(chars), pad_token=".", unk_token="@", begin_seq_token="<", end_seq_token=">")
nationality_vocab = Vocab(sorted(df.nationality))



## Vectorizer

* `vectorizer(tokens)` should return long tensor (vector). Vector values corresponds to tokens. Vector should be filled with padding indexes to satisfy vector maximal size.  

* 👍  method `vectorize(tokens, seq=True)` receives 
    * `tokens` - a list of vocabulary entities, and
    * `seq` - if set to true, then resulting vector represents a sequence.

Let 0 is padding index, 2 is begin of sequence index and 3 is end of sequence index and maximal size is 10. Then for tokens whose indices are, for example, 56, 96 41, a resulting vector should be `[2 56 96 41 3 0 0 0 0]`.  
If `seq` is set to false, resulting vector should be `[56 96 41 0 0 0 0 0 0]`.

In [6]:
class Vectorizer():

    def __init__(self, vocabulary, max_size=-1):
        self.vocab = vocabulary
        self.max_size = max_size

    def vectorize(self, tokens, seq=True):
        
        return


surname_max_size = max(len(surname) for surname in df.surname)
surname_vectorizer = Vectorizer(surname_vocab, surname_max_size + 2)
nationality_vectorizer = Vectorizer(nationality_vocab)

assert nationality_vectorizer.vectorize(["English"], seq=False).tolist() == [4]
assert surname_vectorizer.vectorize("johnson", seq=True).tolist() == [ 2, 43, 48, 41, 47, 52, 48, 47,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0, 0]

## Dataset

`SurnameDataset` class inherits `torch.utils.data.Dataset`.  
Implemented methods are:
* `__init__(df, vectorizer_x, vectorizer_h, nationalities)` initialization receives dataframe `df`, `vectorizer_x` vectorizer for surnames, and `vectorizer_h` for nationalities.
* `set_split()` for setting current data split
* 👍 `__getitem__(idx)` should return triple of vectors: x, y, h where 
    * x is vectorized surname, for example `[2 56 96 41 3 0 0 0 0]`
    * y is x moved to left, for example `[56 96 41 3 0 0 0 0 0]`
    * h is vector for nationality

    

In [7]:
class SurnameDataset(torch.utils.data.Dataset):

    def __init__(self, df, vectorizer_x, vectorizer_h, nationalities=None):
        if nationalities is None:
            self.df = df
        elif isinstance(nationalities, str):
            self.df = df[df.nationality == nationalities]
        else:
            self.df = df[df.nationality.isin(nationalities)]
        self.vectorizer_x = vectorizer_x
        self.vectorizer_h = vectorizer_h
        self._lookup = {split: self.df[self.df.split == split] for split in set(self.df.split)}
        self.set_split("train")
        
    def set_split(self, split):
        self._target_split = split
        self._target_df = self._lookup[split]

    def vectorize_x(self, surname):
        return self.vectorizer_x.vectorize(surname, seq=True)

    def vectorize_y(self, nationality):
        return self.vectorizer_h.vectorize([nationality], seq=False).squeeze()


    def frequency_x(self):
        return torch.tensor([len(self.df[self.df.surname==tok]) for tok in self.vectorizer_x.vocab.ordered_tokens()])

    def frequency_y(self):
        return torch.tensor([len(self.df[self.df.nationality==tok]) for tok in self.vectorizer_h.vocab.ordered_tokens()])

    def __getitem__(self, idx):
        return
        
    def __len__(self):
        return len(self._target_df)

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True):
    for x1, x2, y in torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle):
        yield x1.to(args.device), x2.to(args.device), y.to(args.device)

dataset = SurnameDataset(df, surname_vectorizer, nationality_vectorizer)

assert len(dataset) == 7680
assert len(dataset[0]) == 3

dataset = SurnameDataset(df, surname_vectorizer, nationality_vectorizer, nationalities="English")
assert len(dataset) == 2080
assert len(dataset[0]) == 3

dataset = SurnameDataset(df, surname_vectorizer, nationality_vectorizer, nationalities="Russian")
assert len(dataset) == 1661
assert len(dataset[0]) == 3

dataset = SurnameDataset(df, surname_vectorizer, nationality_vectorizer, nationalities=["English", "Russian"])
assert len(dataset) == 3741
assert len(dataset[0]) == 3


dataset = SurnameDataset(df, surname_vectorizer, nationality_vectorizer)
x, y, h = dataset[0]
assert x.shape == y.shape
assert x.tolist() == [2, 27, 48, 53, 34, 41, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
assert y.tolist()[:-1] == x.tolist()[1:]

# Generator

👍  
`SurnameGenerator` initialization receives 
* `embedding_size` dimension of embedding vector (for surnames)
* `num_embeddings` size of surname vocabulary
* `rnn_hidden_size` dimension of hidden RNN layer
* `num_rnn_hidden_embedding` size of nationality vocabulary
* `dropout_p` probability of dropout

Model will consist of: 
* $E_s$ - embedding layer for surnames, 
* $E_n$ - embedding layer for nationalities,
* GRU - gated reccurent unit
* FC - fully connected layer with dropout

Forward receives 
* $x$ indicies of surnames
* $h$ indicies of nationalityes

then $\hat{y} = FC(GRU(E_s(x), E_n(h)))$.

Apply softmax if `apply_softmax` is set to true.


In [8]:
class SurnameGenerator(torch.nn.Module):
    def __init__(self, embedding_size, num_embeddings, rnn_hidden_size, 
                 num_rnn_hidden_embedding=None, batch_first=True, padding_idx=0, dropout_p=0.5):
        super(SurnameGenerator, self).__init__()


    def forward(self, x, h=None, apply_softmax=False):
        return
        

generator = SurnameGenerator(embedding_size=args.char_embedding_size, 
                             num_embeddings=len(dataset.vectorizer_x.vocab), 
                             rnn_hidden_size=args.rnn_hidden_size,
                             num_rnn_hidden_embedding=len(dataset.vectorizer_h.vocab),
                             batch_first=True,
                             padding_idx=dataset.vectorizer_x.vocab.pad_idx)

x1, x2, h = next(generate_batches(dataset, batch_size=3))
y_hat = generator(x1)
assert y_hat.shape == (3, 19, 80)

y_hat = generator(x1, h)
assert y_hat.shape == (3, 19, 80)

## Settings

In [9]:
# accuracy
def compute_accuracy(y_hat, y):
    _, y_hat_indices = y_hat.max(dim=1)
    n_correct = torch.eq(y_hat_indices, y).sum().item()
    return n_correct / len(y_hat_indices) * 100

# early stopping
def early_stop(train_state, model):
    val_loss = train_state["val_loss"]
    if len(val_loss) < 2:
        torch.save(model.state_dict(), args.model_filename)
        return False
    
    if val_loss[-1] < val_loss[-2]:
        torch.save(model.state_dict(), args.model_filename)
    
    if len(val_loss) >= args.early_stop:
        val_loss =  val_loss[-args.early_stop:]
        return all(val_loss[i] < val_loss[i + 1] 
                   for i in range(args.early_stop - 1))

    return False

# Defining loss function

For 
* $N$ - batch size
* $C$ - sequence size
* $V$ - vocabulary size

let $\hat{y}$ be a prediction tensor of shape $N \times C \times V$ and $y$ be a target tensor of shape $N \times C$.  
Function `compute_loss(y_hat, y)` is responsible for computing negative log-likelihood loss for each datapoint in the batch.

Before applying pyTorch's NLLLoss, each sequence in the batch $\hat{y}$ has to be turned into log of probabilities, i.e. $log(softmax(\hat{y}_i))$ for $i=1...N$. After calculating all $N$ losses by $NLLLoss(log(softmax(\hat{y}_i)), y_i)$ `compute_loss()` returns their mean.




In [10]:
loss_func = torch.nn.NLLLoss(ignore_index=0)

def compute_loss(y_hat, y):
    return

batch_size = 3
seq_size = 2
vocab_size = 4

torch.manual_seed(42)
y_hat = torch.rand(batch_size, seq_size, vocab_size)
y = torch.tensor([[0, 1], [2, 1], [3, 0]])
loss = compute_loss(y_hat, y)
assert torch.all(loss == torch.tensor(1.33540785))


# Training loop

In [ ]:
def compute_accuracy(y_hat, y):
    _, y_hat_indices = y_hat.max(dim=-1)
    y_hat_indices = y_hat_indices.ravel()
    y = y.ravel()
    n_correct = torch.eq(y_hat_indices, y).sum().item()
    return n_correct / len(y_hat_indices) * 100    

# generator

generator = SurnameGenerator(embedding_size=args.char_embedding_size, 
                             num_embeddings=len(dataset.vectorizer_x.vocab), 
                             rnn_hidden_size=args.rnn_hidden_size,
                             num_rnn_hidden_embedding=len(dataset.vectorizer_h.vocab),
                             batch_first=True,
                             padding_idx=dataset.vectorizer_x.vocab.pad_idx)

# seed
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

# loss, optimizer, scheduler
loss_func = torch.nn.NLLLoss(ignore_index=dataset.vectorizer_x.vocab.pad_idx)
optimizer = torch.optim.Adam(generator.parameters(), lr=args.learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)

# progress bars
epoch_bar = tqdm(desc='epochs', total=args.num_epochs, position=0)
dataset.set_split('train')
train_bar = tqdm(desc='train', total=dataset.get_num_batches(args.batch_size), position=1, leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='val', total=dataset.get_num_batches(args.batch_size), position=1, leave=True)

# train state tracker
train_state = {"train_loss": [],
               "train_acc": [],
               "val_loss": [],
               "val_acc": [],}


generator = generator.to(args.device)
try:
    for epoch_index in range(args.num_epochs):
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size)
        running_loss = running_acc = 0.0
        
        generator.train()
        for batch_index, (x, y, h) in enumerate(batch_generator):
            optimizer.zero_grad()
            y_hat = generator(x, h)
            loss = compute_loss(y_hat, y)
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            loss.backward()
            optimizer.step()

            acc_t = compute_accuracy(y_hat, y)
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)        

        # Iterate over val dataset
        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size)
        running_loss = running_acc = 0.0
        
        generator.eval()
        for batch_index, (x, y, h) in enumerate(batch_generator):
            y_hat =  generator(x, h)

            loss = compute_loss(y_hat, y)
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            acc_t = compute_accuracy(y_hat, y)
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            val_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)   

        if early_stop(train_state, generator):
            print("Early stopping")
            break
        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

except KeyboardInterrupt:
    print("Exiting loop")


In [ ]:
plt.plot(train_state["val_loss"])
plt.plot(train_state["train_loss"])

# Testing

In [ ]:
generator.load_state_dict(torch.load(args.model_filename))

generator = generator.to(args.device)
loss_func = torch.nn.NLLLoss()

dataset.set_split('test')
batch_generator = generate_batches(dataset, batch_size=args.batch_size)

running_loss = 0.
running_acc = 0.

generator.eval()
for batch_index, (x, y, h) in enumerate(batch_generator):
    y_hat =  generator(x, h)
    
    # compute the loss
    loss = compute_loss(y_hat, y)
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_hat, y)
    running_acc += (acc_t - running_acc) / (batch_index + 1)

print(f"Test loss: {running_loss:.4f}")
print(f"Test Accuracy: {running_acc:.4f}")

## Sampling

Function `sample_from_model(model, vectorizer, num_samples=10, nationality_idx=None)` must generate `num_samples` surnames. If `nationality_idx` is set to some nationality index, then generated surnames belong to specific nationality. Nationality is represented as first hidden input $h_0$ to GRU.

In [ ]:
def sample_from_model(model, vectorizer, num_samples=10, nationality_idx=None):
    return []

for nationality in nationality_vocab:
    print(nationality)
    samples = sample_from_model(generator, surname_vectorizer, num_samples=3, nationality_idx=nationality_vocab[nationality])
    for sample in samples:
        print(" -", sample)

# Misc

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, svd_solver='full')

emb = generator.emb.weight.data[1:,:]
labels = list(surname_vocab.ordered_tokens())[1:]
x = torch.tensor(pca.fit_transform(emb))


plt.axis("off")
plt.axis([torch.min(x).item(), torch.max(x).item(), torch.min(x).item(), torch.max(x).item()])
for (xi, yi), lbl in zip(x, labels):
    plt.text(xi, yi, lbl)